# Я-Карты (геокодирование) - координаты по адресу

По почтовым адресам определяем координаты объектов (геокодирование). Используем эти координаты для создания сегментов пользователей.
Сегменты пользователей создаем в системах «Яндекс-Аудитории» и «В контакте»

In [1]:
import pandas as pd
import time
import json
import requests

# Вывод всех операций в Jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Подключение библиотеки ГЕОКОДИРОВАНИЯ 

# pip install yandex-geocoder
from decimal import Decimal
from yandex_geocoder import Client

In [90]:
# Личный кабинет на Яндекс-картах
# https://developer.tech.yandex.ru/services/abs3/

# API - KEY от Яндекс-карт
client = Client("********************************")

#### Параметры

In [4]:
# Путь к файлам
path = "csv/"
# Имена файлов
file_name_from = "coordinates_from.csv"
file_name_to = "coordinates_to.csv"

# Радиус в метрах
RADIUS = 500

#### Пример

In [5]:
# Получение координат по адресу
# coordinates = client.coordinates("Москва Льва Толстого 16")

# Получение АДРЕСА по координатам
# address = client.address(Decimal("37.587093"), Decimal("55.733969"))

In [44]:
def result_to_df(lst):
    # https://ru.stackoverflow.com/questions/1413824/%D0%9F%D1%80%D0%B5%D0%BE%D0%B1%D1%80%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D1%8C-%D0%B2%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9-%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C-%D0%B2-dataframe
    """
    Преобразуем lst (в виде списка МНОГОУРОВНЕВЫХ словарей) в Датафрейм
    Название колонки формируется АВТОМАТИЧЕСКИ - наименование ключа.
    Если словарь вложенный - составные ключи через точку
    Входные данные: Список словарей
    Выходные данные: Датафрейм
    """

    df = pd.DataFrame()
    for adds in lst:
        df = pd.concat([df, pd.json_normalize(adds)], ignore_index=True)
        
    return df

# Шаг. Считываем адреса - определяем координаты

In [114]:
df = pd.read_csv(path+file_name_from, sep=";")
df.head(5)

,region,adress
0,Москва,"Россия, г Москва, (м Сокол) пр-кт Ленинградски..."
1,Москва,"Россия, г Москва, (м Серпуховская) ул Люсиновс..."
2,Москва,"Россия, г Москва, (м Павелецкая) ул Валовая, д..."
3,Москва,"Россия, г Москва, (м Жулебино) ул Тарханская, д 1"
4,Москва,"Россия, г Москва, (м Академическая) ул Новочер..."


In [195]:
def get_coordinates_from_address(df, file_name_to):
    """
    Функция: 
    - считывает из Входного ДатаФрейма адреса
    - Преобразует их в координаты
    - Координаты  преобразует в адреса (для проверки)
    - Записывает это в ввыходной Датафрейм
    
    Входные данныеЖ
    - Датафрейм
    - Имя выходного датафрейма
    
    На выходе - сложный словарь
   
    """
    
    
    # coordinates - координаты адреса
    # adress_back - получение адреса по координатам (для проверки)
    df['coordinates'] = ""
    df['adress_back'] = ""

    for index, row in df.iterrows():
        # Получаем координаты     
        coordinates = client.coordinates(row['adress'])
        row['coordinates'] = coordinates
        # Получаем адрес по координатам
        adress_back = client.address(coordinates[0], coordinates[1])
        row['adress_back'] = adress_back

    # Сохраняем датаФрейм в файле
    df.to_csv(file_name_to, sep=";", index=False, encoding='utf-8-sig')
    
    
    # ***
    # *** Формируем из Датафрейма Словарь по каждому региону
    dic_coordinates = {}

    for index, row in df.iterrows():
        # Получаем координаты     
        region = row['region']
        adress = row['adress']
        coordinates = row['coordinates']
        adress_back = row['adress_back']
        param_current = {'region':region, 'adress':adress, 'coordinates':coordinates, 'adress_back':adress_back}

        if dic_coordinates.get(region):
            # По этому региону уже есть одна точка
            dic_coordinates[region]['param'].append(param_current)
        else:
            # Словарь по региону
            dic_coordinates[region] = {}
            dic_coordinates[region]['param'] = [param_current]

    return dic_coordinates

# Шаг. Формируем словарь с параметрами

In [117]:
# Формируем из Датафрейма Словарь по каждому региону
dic_coordinates = {}

for index, row in df.iterrows():
    # Получаем координаты     
    region = row['region']
    adress = row['adress']
    coordinates = row['coordinates']
    adress_back = row['adress_back']
    param_current = {'region':region, 'adress':adress, 'coordinates':coordinates, 'adress_back':adress_back}
    
    if dic_coordinates.get(region):
        # По этому региону уже есть одна точка
        dic_coordinates[region]['param'].append(param_current)
    else:
        # Словарь по региону
        dic_coordinates[region] = {}
        dic_coordinates[region]['param'] = [param_current]

dic_coordinates

{'Москва': {'param': [{'region': 'Москва',
    'adress': 'Россия, г Москва, (м Сокол) пр-кт Ленинградский, д 77 к 1',
    'coordinates': (Decimal('37.509289'), Decimal('55.804901')),
    'adress_back': 'Россия, Москва, Ленинградский проспект, 77к1'},
   {'region': 'Москва',
    'adress': 'Россия, г Москва, (м Серпуховская) ул Люсиновская, д 48-50 к 10',
    'coordinates': (Decimal('37.621579'), Decimal('55.721077')),
    'adress_back': 'Россия, Москва, Люсиновская улица, 48-50к10'},
   {'region': 'Москва',
    'adress': 'Россия, г Москва, (м Павелецкая) ул Валовая, д 8/18',
    'coordinates': (Decimal('37.632125'), Decimal('55.730887')),
    'adress_back': 'Россия, Москва, Валовая улица, 8/18'},
   {'region': 'Москва',
    'adress': 'Россия, г Москва, (м Жулебино) ул Тарханская, д 1',
    'coordinates': (Decimal('37.858581'), Decimal('55.685406')),
    'adress_back': 'Россия, Москва, Тарханская улица, 1'},
   {'region': 'Москва',
    'adress': 'Россия, г Москва, (м Академическая) ул Но

# Шаг. Я-Аудитории

In [9]:
from lib_yaaudience.__init__ import *
from lib_yaaudience.client import *
from lib_yaaudience.core import *
from lib_yaaudience.objects import *

In [10]:
# Токен от Яндекс_метрики
token_YM = '********************************'

headers = {
    "Host": "api-audience.yandex.ru",
    "Authorization": "OAuth " + token_YM,
    "Content-Type": "application/json",
    "Content-Length": "123"
}

# Создание объекта для оберточной API
ya = YaAudience(token=token_YM)

#### Список сегментов

In [209]:
ya.segments()

[-----------------------------
 
 <SegmentObject{'id': 24396340, 'name': 'test____segment_csv_01', 'type': 'uploading', 'status': 'few_data', 'create_time': datetime.datetime(2022, 7, 14, 15, 59, 34, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'can_create_dependent': False, 'has_derivatives': False, 'cookies_matched_quantity': 2, 'hashed': False, 'content_type': 'crm', 'item_quantity': 188, 'valid_unique_quantity': 188, 'valid_unique_percentage': '100%', 'matched_quantity': 1, 'matched_percentage': '0.53%', 'counter_id': 0, 'guest': False}>
 
 -----------------------------,
 -----------------------------
 
 <SegmentObject{'id': 24733284, 'name': 'test____segment_from_metrika_02', 'type': 'metrika', 'status': 'processed', 'create_time': datetime.datetime(2022, 7, 20, 8, 25, 28, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'c

#### Функция - Создать сегмент типа "геолокация-окружность"

In [97]:
def create_geo(points, segment_name, period_length, times_quantity, radius=RADIUS):
    # https://yandex.ru/dev/audience/doc/segments/creategeo.html
    """
    Входные данные:
    - points - cписок точек в виде словарей
    - segment_name - наименование сегмента
    - period_length - Период посещений указанных мест в днях
    - times_quantity - частота посещений за период
    """
  

    URL = "https://api-audience.yandex.ru/v1/management/segments/create_geo?"

    body = {}
    body['segment'] = {}
    body['segment']['name'] = segment_name
    # Не менее 500 метров, иначе - impossible to create geo segment with radius that is less than 500
    body['segment']['radius'] = radius
    body['segment']['points'] = points

    # Период посещений указанных мест. Параметр необходим для создания условия «Пользователь посетил указанные места 
    # N раз за период». Допустимые значения периода (в сутках): от 1 до 90 (включительно).
    body['segment']['period_length'] = period_length

    # Частота посещений указанных мест. Параметр необходим для создания условия «Пользователь посетил указанные места N раз за период».
    # Примечание: Учитывается только одно посещение в сутки
    body['segment']['times_quantity'] = times_quantity

    # Тип координат для создания сегмента.
    # last — актуальные координаты. В сегмент попадут пользователи, которые находятся на выбранной территории прямо сейчас, 
    #         или находились там не более часа назад.
    # regular — регулярные координаты. В сегмент попадут пользователи, которые регулярно бывают на выбранной территории. 
    #     Например, живут или работают рядом. При подборе учитываются данные за последние 45 дней.
    # home — координаты дома. В сегмент попадут пользователи, которые живут на выбранной территории.
    # work — координаты работы. В сегмент попадут пользователи, которые работают на выбранной территории.
    # condition — условие «Пользователь посетил указанные места N раз за период». 
    #     В сегмент попадут пользователи, которые выполнили заданное условие.
    body['segment']['geo_segment_type'] = 'condition'

    # конвертируем словарь в JSON-формат и перекодируем в UTF-8
    body = json.dumps(body, ensure_ascii=False).encode('utf8')

    req = requests.post(URL, body, headers=headers)
    return req, req.text

#### Создаем сегменты типа "геолокация-окружность" по каждому городу (1 город - 1 сегмент)

In [200]:
def ya_creating_segments_by_city(dic_coordinates, radius, period_length, times_quantity):
    # Готовим массив данных для Я-Аудиторий из нашего словаря и создаем сегменты
    # Радиус в метрах


    for segment_name in dic_coordinates:
        # Инициируем создание массива координат
        points = []
        print(segment_name)
        info = dic_coordinates[segment_name]['param']
        for el in info:
            # Новая координата
            current_point = {}
            current_point['latitude'] = float(el['coordinates'][0])
            current_point['longitude'] = float(el['coordinates'][1])
            points.append(current_point)

        print(points)
        req, req_text = create_geo(points, segment_name, period_length, times_quantity, radius=radius)
        print(req)
        print(req_text)

# Шаг. VK

In [40]:
import vk_api

In [203]:
# Остапа
TOKEN = '***************************************************'
ACCOUNT_ID = 111111111111 
CLIENT_ID = 22222222222

In [204]:
params_base = {'account_id': ACCOUNT_ID}
session = vk_api.VkApi(token=TOKEN)

In [208]:
def getAdsTargeting(params):
    """
    https://vk.com/dev/ads.getAdsTargeting
    возвращает параметры таргетинга РО
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getAdsTargeting", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 
# фильтр по рекламным кампаниям.
params['campaign_ids'] = "[1024581970]"
result, df_result = getAdsTargeting(params)
df_result

,id,campaign_id,age_from,age_to,geo_near,price_list_audience_type,count,geo_point_type
0,126933804,1024581970,14,80,"55.804901,37.509289,1000;55.721077,37.621579,1...",0,260000,regular


#### Функция - смены параметров РО

In [68]:
def updateAds(params):
    """
    https://vk.com/dev/ads.updateAds
    Меняем  список рекламных объявлений
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.updateAds", params)
    df = result_to_df(result)
    
    return result, df

In [191]:
# Готовим данные для ОБЪЯВЛЕНИЯ
# Возьмем коородинаты по 1-му городу
RADIUS = 1000

gorod_1 = dic_coordinates['Москва']
param = gorod_1['param']

geo_near = ""
for point in param:
    geo_near +=  f"{float(point['coordinates'][1])},{float(point['coordinates'][0])},{RADIUS};"
    
geo_near = geo_near[:-1]
geo_near    

'55.804901,37.509289,1000;55.721077,37.621579,1000;55.730887,37.632125,1000;55.685406,37.858581,1000;55.681849,37.582996,1000;55.723972,37.437182,1000;55.871011,37.682916,1000;55.761771,37.40866,1000'

#### Вызываем функцию - смена параметров объявления (меняем гео таргетинг)

In [192]:
params = params_base.copy()

ad_01 = {}
# Идентификатор редактируемого объявления.
ad_01['ad_id'] = 126933804

ad_01['geo_near'] = geo_near
# Тип мест гео-таргетинга. Возможные значения: regular — Регулярно бывает; home — Дом; work — Работа или учёба;
# ad_01['geo_point_type'] = "regular"
ad_01['age_from'] = 14
ad_01['age_to'] = 80
# Пол - любой
ad_01['sex'] = 0

data = f"[{ad_01}]"
data = data.replace('\'', '"')   # Иначе - не получится
params["data"] = data


result, df_result = updateAds(params)
result

[{'id': 126933804}]

# VK-создание АУДИТОРИИ  ретаргетинга

### ads.getTargetGroups- Возвращает список аудиторий ретаргетинга

In [194]:
def getTargetGroups(params):
    """
    https://vk.com/dev/ads.getTargetGroups
    Возвращает список аудиторий ретаргетинга
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getTargetGroups", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 

result, df_result = getTargetGroups(params)
df_result

,id,name,last_updated,audience_count,lifetime,is_audience,is_shared,file_source,api_source,lookalike_source
0,48652446,phone_500,1659636731,970,720,True,False,True,True,False
1,48652574,phone_100000,1659594635,99000,720,True,False,True,False,False
2,48665906,Null segment,1659617143,520,720,True,False,False,True,False
3,48673250,Lookalike-аудитория 4 авг,0,0,720,True,False,False,False,False
4,48682492,5 августа,1659683209,570,720,True,False,False,True,False
5,48746245,8 августа,0,0,720,True,False,False,False,False
6,48746571,8 августа_02,0,0,720,True,False,False,False,False
7,48746582,8 августа_50000,0,0,720,True,False,False,False,False
8,48746591,8 августа_500,0,0,720,True,False,False,False,False
9,48749167,8 августа_1000_plus,0,0,720,True,False,False,False,False


#### ads.createTargetGroup - Создаёт аудиторию

In [193]:
def createTargetGroup(params):
    """
    https://vk.com/dev/ads.createTargetGroup
    Создаёт аудиторию для ретаргетинга рекламных объявлений на пользователей, которые посетили сайт рекламодателя 
    (просмотрели информации о товаре, зарегистрировались и т.д.)
    
    Остап: можно создать "пустую" аудиторию, а затем с помощью метода "importTargetContacts" - добавить туда контакты!!!
    
    В результате выполнения метода возвращается идентификатор созданной аудитории.
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.createTargetGroup", params)
    
    return result

params = params_base.copy()
# Наименование АУДИТОРИИ
params['name'] = "8 августа_1000_plus"

ad_0 = {}
ad_0['geo_near'] = geo_near
data = f"[{ad_0}]"
data = data.replace('\'', '"')   # Иначе - не получится
params["data"] = data

result = createTargetGroup(params)
result

params

{'id': 48749167}

{'account_id': 1607796571,
 'name': '8 августа_1000_plus',
 'data': '[{"geo_near": "55.804901,37.509289,1000;55.721077,37.621579,1000;55.730887,37.632125,1000;55.685406,37.858581,1000;55.681849,37.582996,1000;55.723972,37.437182,1000;55.871011,37.682916,1000;55.761771,37.40866,1000"}]'}

# ===============================================
# Создание сегмантов по списку Адресов (Я-А, VK)
# Итоговый код
# ===============================================

In [207]:
# Путь к файлам
path = "csv/"
# файл с координатами
file_name_from = "coordinates_from.csv"
# файл с результатами геокодирования
file_name_to = "coordinates_to.csv"

# Считываем адреса
df = pd.read_csv(path+file_name_from, sep=";")

# Формируем словарь с координатами + записываем ДаиаФрейм для проверки
coordinates_dic = get_coordinates_from_address(df, path+file_name_to)

# ***
# *** Создаем сегменты в Яндекс-Аудиториях *****************************************************************
print("************************ Сегменты в Яндекс-Аудиториях ***************************")
print()

# Радиус в метрах
radius = 500
period_length = 10
times_quantity = 1
# Функция создания сегментов
ya_creating_segments_by_city(coordinates_dic, radius, period_length, times_quantity)



# ***
# *** Создаем ГЕО-Таргетинг в VK для отдельного объявления *************************************************
print("************************ Таргетинг в VK ***************************")

# Готовим данные для ОБЪЯВЛЕНИЯ
# Возьмем коородинаты по 1-му городу

# Идентификатор объявления
Ad_id = 126933804
RADIUS = 1000
name_geo = 'Москва'

gorod_1 = coordinates_dic[name_geo]
param = gorod_1['param']

geo_near = ""
for point in param:
    geo_near +=  f"{float(point['coordinates'][1])},{float(point['coordinates'][0])},{RADIUS};"
    
geo_near = geo_near[:-1]
params = params_base.copy()
ad_01 = {}
ad_01['ad_id'] = Ad_id
ad_01['geo_near'] = geo_near

data = f"[{ad_01}]"
data = data.replace('\'', '"')   # Иначе - не получится
params["data"] = data

result, df_result = updateAds(params)
result

************************ Сегменты в Яндекс-Аудиториях ***************************

Москва
[{'latitude': 37.509289, 'longitude': 55.804901}, {'latitude': 37.621579, 'longitude': 55.721077}, {'latitude': 37.632125, 'longitude': 55.730887}, {'latitude': 37.858581, 'longitude': 55.685406}, {'latitude': 37.582996, 'longitude': 55.681849}, {'latitude': 37.437182, 'longitude': 55.723972}, {'latitude': 37.682916, 'longitude': 55.871011}, {'latitude': 37.40866, 'longitude': 55.761771}]
<Response [200]>
{"segment":{"id":25663474,"name":"Москва","type":"geo","status":"is_processed","create_time":"2022-08-09T10:04:24+03:00","owner":"smbz-demis","has_guests":false,"guest_quantity":0,"can_create_dependent":false,"has_derivatives":false,"geo_segment_type":"condition","times_quantity":1,"period_length":10,"form":"circle","radius":500,"points":[{"latitude":37.509289,"longitude":55.804901},{"latitude":37.621579,"longitude":55.721077},{"latitude":37.632125,"longitude":55.730887},{"latitude":37.858581,"lo

[{'id': 126933804}]